## Agents
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

In [2]:
# openai key is loaded in .env file. load the environment variables
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Tool calling agent
Tool calling allows a model to detect when one or more tools should be called and respond with the inputs that should be passed to those tools. 

Use case: Let's load certificate pdf, mongodb certificate and query on those pdfs.
Certificates and MongoDB pdf files are prety big. So we need to break certificate and mongodb files into pages.

In [3]:
# https://www.openmymind.net/mongodb.pdf
# https://www.first.org/resources/guides/Establishing-a-Certification-Authority-CA.pdf
from langchain_community.document_loaders import PyPDFLoader

# load multiple pdf docs
# Load the PdF document using langchain_community.document_loaders import PyPDFLoader
certs_doc_loader = PyPDFLoader("https://www.first.org/resources/guides/Establishing-a-Certification-Authority-CA.pdf")
certs_doc_pages = certs_doc_loader.load_and_split()

# loag 
# mongodb loader
mongodb_doc_loader = PyPDFLoader("https://www.openmymind.net/mongodb.pdf")
mongodb_doc_pages = mongodb_doc_loader.load_and_split()

Embedding: Let's load embedding model

In [4]:
# create embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Users/aryanraghav/workspace/langchain-learning/langchain-examples/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aryanraghav/workspace/langchain-learning/langchain-examples/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Create Index: Load pages with embedding to Vector store

In [6]:
from langchain_community.vectorstores import FAISS

# create a vector store
certs_vector = FAISS.from_documents(certs_doc_pages, embeddings)
mongodb_vector = FAISS.from_documents(mongodb_doc_pages, embeddings)

Retriever: Create retriever for certificate and mongodb index

In [8]:
certs_retriver = certs_vector.as_retriever(search_type="similarity", search_kwargs={"k":4})
# certs_response = certs_retriver.invoke("What is a Certification Authority?")

mongodb_retriver = mongodb_vector.as_retriever(search_type="similarity", search_kwargs={"k": 4})
# mongodb_response = mongodb_retriver.invoke("What is MongoDB?")

In [ ]:
Create Tool: Create retriever as tool for agents

In [9]:
# crate certs_retriver and mongodb_retriver
from langchain.tools.retriever import create_retriever_tool

certs_retriver_tool = create_retriever_tool(
    certs_retriver,
    "search_for_certification_details",
    "Searches and returns information about the certification in the document.",
)

mongodb_retriver_tool = create_retriever_tool(
    mongodb_retriver,
    "search_for_mongodb_details",
    "Searches and returns information about MongoDB in the document.",
)

Create Agent with tools

In [10]:
tools = [certs_retriver_tool, mongodb_retriver_tool]

# create agent
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain import hub
from os import getenv

# llm
llm_model = ChatOpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

Agent is ready for quering documents

In [12]:
# Query certificates pdf
certs_response = agent_executor.invoke({"input": "what is certificates?"})
print(certs_response['output'])

Certificates are an essential part of a Certification Authority (CA) system. A Certificate Management System is used for maintaining all certificates, including creating, suspending, renewing, revoking, and deploying certificates. Subscribers are individuals or entities whose names appear as the subject in a certificate, and they assert the use of the key and certificate according to the Certificate Policy. Relying Parties rely on the validity of the binding of the Subscriber's name to a public key for various purposes.

There are different components in a PKI ecosystem, such as Certification Authorities (CAs), Registration Authorities (RAs), and Trust Stores. CAs are responsible for signing digital certificates, which can be public CAs found on the Internet or private (internal) CAs located within organizations for secure operations. RAs act as the point of contact for individuals who want to get their certificate signed or revoked by the CA, and they verify the authenticity of the re

In [13]:
# Query MongoDB pdf
mongodb_response = agent_executor.invoke({"input": "what is Mongodb?"})
print(mongodb_response['output'])

MongoDB is a popular NoSQL database that is schema-less, meaning a collection isn't strict about what goes in it. Fields are tracked with each individual document. MongoDB uses a binary serialized JSON format called BSON. It allows for easy insertion of documents into collections using commands like `insert` and `find`. Each document must have a unique `_id` field, which can be generated by MongoDB or by the user. MongoDB offers powerful aggregation capabilities through features like Aggregation Pipeline and MapReduce. If you want to learn more about MongoDB, you can refer to resources like "The Little MongoDB Book" available on GitHub.


In [14]:
# Query both certificates and MongoDB pdf
certs_mongodb_response = agent_executor.invoke({"input": "what is Mongodb? What is certificate? How certificates are used in MongoDB?"})
print( certs_mongodb_response["output"])

### MongoDB:
MongoDB is a popular NoSQL database management system. It is a document-oriented database, classified as a NoSQL database program that uses JSON-like documents with optional schemas.

### Certificate:
A certificate is a digital certificate, also known as a public key certificate, used to cryptographically link ownership of a public key with the entity that owns it. Digital certificates are used for sharing public keys to be used for encryption and authentication. Certificates are digitally signed by a Certification Authority (CA) and conform to the X.509 v3 standard.

### How certificates are used in MongoDB:
In MongoDB, certificates are used for authentication and encryption purposes. Certificate Management Systems are used to maintain all certificates, create, suspend, renew, revoke, and deploy certificates. Subscribers in MongoDB are entities whose names appear as the subject in a certificate, and Relying Parties rely on the validity of the binding of the Subscriber's n